In [1]:
!pip install levenshtein sentence-transformers nltk -q

from typing import List, Tuple
from statistics import mean
from Levenshtein import ratio as levenshtein_ratio
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import json

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━

2025-08-05 06:19:42.832132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754374783.094952      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754374783.177137      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def evaluate_metrics(references: List[str], predictions: List[str]) -> dict:
    assert len(references) == len(predictions), "Lists must be of equal length."

    levenshtein_scores , cosine_scores = [] , []
    for ref, pred in zip(references, predictions):
        
        # Levenshtein Ratio
        lev = levenshtein_ratio(ref, pred)
        levenshtein_scores.append(lev)
        
        # Cosine Similarity
        embeddings = sbert_model.encode([ref, pred], convert_to_tensor=True)
        cosine = util.cos_sim(embeddings[0], embeddings[1]).item()
        cosine_scores.append(cosine)
    
    def summarize(scores: List[float]) -> Tuple[float, float, float]:
        return round(mean(scores), 4), round(max(scores), 4), round(min(scores), 4)
    
    return {
        "Levenshtein Ratio": summarize(levenshtein_scores),
        "Cosine Similarity": summarize(cosine_scores)
    }

In [7]:
f32  = pd.read_parquet("/kaggle/input/outputsandlogs/float32.parquet")
bf16 = pd.read_parquet("/kaggle/input/outputsandlogs/bfloat16.parquet")
with open("/kaggle/input/miccaireg/labels.json") as f: ground = json.load(f)[4900:]

outputs_f32 = [f32.iloc[i]["generated_text"] for i in range(len(f32))]
outputs_bf16 = [bf16.iloc[i]["generated_text"] for i in range(len(bf16))]

In [8]:
len(outputs_f32) , len(outputs_bf16) , len(ground)

(209, 209, 209)

In [9]:
%%capture

bf16_results = evaluate_metrics(ground, outputs_bf16)
f32_results  = evaluate_metrics(ground, outputs_f32)

In [10]:
for label, results in [("BF16", bf16_results), ("FP32", f32_results)]:
    print(f"\n=== {label} Results ===")
    for metric, (mean_val, max_val, min_val) in results.items():
        print(f"{metric}: mean={mean_val}, max={max_val}, min={min_val}")


=== BF16 Results ===
Levenshtein Ratio: mean=0.2536, max=0.4839, min=0.081
Cosine Similarity: mean=0.4128, max=0.7669, min=0.2039

=== FP32 Results ===
Levenshtein Ratio: mean=0.2672, max=0.597, min=0.0818
Cosine Similarity: mean=0.4866, max=0.8552, min=0.1711
